In [1]:
from __future__ import annotations

import pprint
import re
import struct
from array import array
from dataclasses import dataclass
from enum import Enum
from pathlib import Path
from typing import BinaryIO
from warnings import warn

## Shared / library code

In [2]:
# Why isn't this a built-in function already?

def unpack_stream(format: str, stream: BinaryIO):
    size = struct.calcsize(format)
    buf = stream.read(size)
    assert len(buf) == size
    assert isinstance(buf, bytes)
    return struct.unpack(format, buf)    

In [3]:
class HoverRaceTrackException(Exception):
    pass

In [4]:
# TODO: write methods or functions to read/write from/to a string
# https://github.com/HoverRace/HoverRace/blob/master/engine/Parcel/ClassicObjStream.cpp

MAX_STRING_LEN = 16 * 1024

def pack_string_length(size: int) -> bytes:
    if size < 0xff:
        return struct.pack('<B', size)
    elif size < 0xfffe:
        return struct.pack('<BH', 0xff, size)
    else:
        return struct.pack('<BHL', 0xff, 0xffff, size)

def unpack_string_length(stream: BinaryIO) -> int:
    (b,) = unpack_stream('<B', stream)
    if b < 0xff:
        return b
    (w,) = unpack_stream('<H', stream)
    if w == 0xfffe:
        # 0xfffe is the marker for Unicode strings.
        raise NotImplementedError('ClassicObjStream::ReadStringLength for unicode strings')
    elif w == 0xffff:
        (dw,) = unpack_stream('<L', stream)
        return dw
    else:
        return w

def pack_string(s: str) -> bytes:
    b = s.encode('ascii')
    size = len(b)
    if size > MAX_STRING_LEN:
        # Note: this logic will lead to broken last character if we use utf-8.
        warn('String length {} exceeds max {}; truncated: {}...'.format(size, MAX_STRING_LEN, s[:64]))
        size = MAX_STRING_LEN
    return pack_string_length(size) + b[:size]

def unpack_string(stream: BinaryIO) -> str:
    size = unpack_string_length(stream)
    excess = 0
    if size > MAX_STRING_LEN:
        warn('String length {} exceeds max {}; truncating'.format(size, MAX_STRING_LEN))
        excess = size - MAX_STRING_LEN
        size = MAX_STRING_LEN
    (buf,) = unpack_stream('{}s{}x'.format(size, excess), stream)
    return buf.decode('ascii')

In [5]:
# https://github.com/HoverRace/HoverRace/blob/master/engine/Parcel/ClassicRecordFile.cpp

class ClassicRecordFileHeader:
    def __init__(self, *, title='', sumValid=False, checksum=0, recordsMax=0):
        self.title = title
        self.sumValid = sumValid
        self.checksum = checksum
        self.recordsUsed = 0
        assert recordsMax >= 0
        self.recordsMax = recordsMax # AKA numRecords
        # recordList should be a list of four UInt32
        self.recordList = array('L')
        self.recordList.extend(0 for i in range(recordsMax))

    def __repr__(self):
        return re.sub(r'\s+', ' ', '''
        ClassicRecordFileHeader(
            title={self.title!r},
            sumValid={self.sumValid!r},
            checksum={self.checksum!r},
            recordsMax={self.recordsMax!r},
        )'''.format(self=self)).strip()

    # AKA .Serialize() for writing
    def pack(self) -> bytes:
        raise NotImplementedError('WIP')
        return struct.pack()

    # AKA .Serialize() for reading
    # Modifies in-place
    def unpack_here(self, stream: BinaryIO):
        self.title = unpack_string(stream)
        self.sumValid, self.checksum, self.recordsUsed, self.recordsMax = unpack_stream('<8xLLLL8x', stream)
        self.sumValid = bool(self.sumValid)
        self.recordList = array('L')

        assert self.recordsMax == 4

        # Check title for validity.
        if 'HoverRace track file' not in self.title and 'Fireball object factory resource file' not in self.title:
            raise HoverRaceTrackException('Missing or corrupt header')

        for i in range(self.recordsMax):
            # This is a list of "pointers" to file positions.
            (uint32,) = unpack_stream('<L', stream)
            self.recordList.append(uint32)
            # https://github.com/HoverRace/HoverRace/blob/master/doc/dev/track-format.txt
            # A track has 4 records.
            # Record 0 is a TrackEntry, AKA metadata.
            # Record 1 is a Level.
            # Record 2 is a Background (optional?). [NOT a background palette (8128 * 3 bytes)?]
            # Record 3 is the map (LoadMap), AKA a texture. (optional?)

    # AKA .Serialize() for reading
    # Returns a new instance.
    @classmethod
    def unpack(cls, stream: BinaryIO) -> ClassicRecordFileHeader:
        obj = cls()
        obj.unpack_here(stream)
        return obj

In [6]:
# https://github.com/HoverRace/HoverRace/blob/master/engine/Parcel/ClassicRecordFile.cpp

class ClassicRecordFile:
    def __init__(self):
        self.header = None
        self.entry = None

    def __repr__(self):
        return '<ClassicRecordFile header={self.header!r} entry={self.entry!r}>'.format(self=self)

    def read(self, stream: BinaryIO):
        self.header = ClassicRecordFileHeader.unpack(stream)
        stream.seek(self.header.recordList[0])
        self.entry = TrackEntry.unpack(stream)
        warn('WIP')

In [7]:
MR_REGISTERED_TRACK = 0;
MR_FREE_TRACK = 1;

class RegistrationMode(Enum):
    REGISTERED = 0
    FREE = 1

In [8]:
# https://github.com/HoverRace/HoverRace/blob/master/engine/Model/TrackEntry.cpp

MR_MAGIC_TRACK_NUMBER = 82617
MR_NOBITMAP = 0;
MR_BITMAP = 1;

@dataclass
class TrackEntry:
    # name: str = ''  # The name is not part of the file. It's likely coming from the filename.
    description: str = ''
    regMinor: int = 0
    regMajor: int = 0
    registrationMode: RegistrationMode = 0
    sortingIndex: int = 0

    @classmethod
    def unpack(cls, stream: BinaryIO):
        obj = cls()
        magicNumber, version = unpack_stream('<ll', stream)
        if magicNumber != MR_MAGIC_TRACK_NUMBER:
            raise HoverRaceTrackException('Bad magic number: 0x%08x' % magicNumber)
        if version != 1:
            raise HoverRaceTrackException('Unknown track version: {}'.format(magicNumber))
        obj.description = unpack_string(stream)
        obj.regMinor, obj.regMajor, obj.sortingIndex, mode = unpack_stream('<llll', stream)
        obj.registrationMode = RegistrationMode(mode)
        if obj.registrationMode == RegistrationMode.FREE:
            (magicNumber2,) = unpack_stream('<l', stream)
            if magicNumber2 != MR_MAGIC_TRACK_NUMBER:
                raise HoverRaceTrackException('Bad magic number for free track: 0x%08x' % magicNumber2)
        return obj


## Main application-like code

In [9]:
BASEDIR = Path('~/.var/app/com.valvesoftware.Steam/data/Steam/steamapps/common/HoverRace/tracks/').expanduser()

In [10]:
with open(BASEDIR / 'ClassicH.trk', 'rb') as f:
    record = ClassicRecordFile()
    record.read(f)

/tmp/ipykernel_1222691/2142615983.py:15: UserWarning: WIP
  warn('WIP')


In [11]:
record

<ClassicRecordFile header=ClassicRecordFileHeader( title='\x08\rHoverRace track file, (c)GrokkSoft 1997\n\x1a', sumValid=True, checksum=2673339213, recordsMax=4, ) entry=TrackEntry(description='Level: Easy\nLength: 0.6 km\nShort and fast track\nLots of curve and no jumps\n\n', regMinor=0, regMajor=0, registrationMode=<RegistrationMode.FREE: 1>, sortingIndex=30)>

In [12]:
[hex(x) for x in record.header.recordList]

['0x5c', '0xc5', '0x1d7b', '0x81eff']